# 🔰 Getting started with cherwell_pydantic_api

## Step 1: create cherwell.env

The `cherwell.env` file contains the URL of your Cherwell API endpoint along with the credentials needed.

Use `cwcli` on the command line to create your `cherwell.env` file, and `cwcli check` to check connectivity.

You can then proceed with this notebook. To get started, you can skip forward to the chapter **[Set up the business object filters](#bo_filter)**.

### Set up the environment and connect to the API

These cells set up the Jupyter notebook environment and the connection to the Cherwell API.

In [ ]:
# The working directory is expected to contain the cherwell.env configuration file.
# This implementation changes the working directory to the parent of the cherwell_pydantic_api package.
# Alter it if your setup is different.

import cherwell_pydantic_api
homedir = cherwell_pydantic_api.__path__[0] + '/../'
%cd $homedir

In [ ]:
# The Interactive module converts async/await calls to sync calls, so that you don't have to type await all the time.
# For this to work under Jupyter Notebook, the nest_asyncio module is required. (IPython on the console doesn't require this.)

import nest_asyncio
nest_asyncio.apply()

%gui asyncio

In [ ]:
# Import some useful classes into the namespace.

from cherwell_pydantic_api.bo.modelgen.repo import ModelRepo
from cherwell_pydantic_api.instance import Instance
from cherwell_pydantic_api.interactive import Interactive
from cherwell_pydantic_api.settings import Settings


# Set up the instance, repo and cw objects

instance = Instance.use()
repo = ModelRepo(create=True)
print(f"base_url={instance.settings.base_url}")

cw = Interactive(waiter=get_ipython().loop_runner)

In [ ]:
# Authenticate to the API. This is the first actual HTTP callout. It might take a while if you are connecting to a sleeping test instance of Cherwell.

print("Authenticating...")
%time cw.authenticate()

## Set up the Collector

The `Collector` object collects Business Object schemas in preparation for Pydantic model generation.

This part of the notebook allows you to interactively choose which schemas you wish to model.

### Setup widgets

In [ ]:
# Prepare some widgets for the datagrids below.

import ipywidgets
from ipydatagrid import DataGrid, VegaExpr, TextRenderer
from pandas import DataFrame

renderer_verdict = TextRenderer(
    background_color = VegaExpr("cell.metadata.data['verdict'] ? 'lightgreen' : ''")
)
renderer_bool = TextRenderer(
    text_value = VegaExpr("""isValid(cell.value) ? (cell.value === true ? 'TRUE' : cell.value === false ? 'false' : cell.value) : '(None)'"""),
    background_color = VegaExpr("isValid(cell.value) ? (cell.value ? 'lightgreen' : 'lightpink') : 'gainsboro'")
)
renderers = {
    'name': renderer_verdict,
    'verdict': renderer_bool,
    'num_fields': renderer_bool,
    'lookup': renderer_bool,
    'major': renderer_bool,
    'supporting': renderer_bool,
}

In [ ]:
# Create the Collector object and async_wrap it in the cw Interactive instance.

from cherwell_pydantic_api.bo.modelgen.collector import Collector

cw.async_wrap(collector=Collector(instance, verbose=False))

In [ ]:
bo_data = None
bo_datagrid = None

################
# Set up widgets

filter_checkbox = ipywidgets.Checkbox(description='Filter by verdict')
def filter_change(change):
    transforms = [{'type': 'sort', 'columnIndex': 1, 'desc': False}]
    if filter_checkbox.value:
        transforms.append({"type": "filter", "columnIndex": 2, "operator": "in", "value": [True]})
    bo_datagrid.transform(transforms)
filter_checkbox.observe(filter_change, names='value')
status_output = ipywidgets.Output(layout={'border': '2px solid orange'})

################
# Run the collector and set up the datagrid if it doesn't already exist, otherwise update it (reduce flickering)

def collect(include_filter, exclude_filter):
    global bo_data, bo_datagrid, status_output
    cw.collector.bo_include_filter = include_filter
    cw.collector.bo_exclude_filter = exclude_filter
    print("Collector loading data...")
    cw.collector.collect()
    print(f"OK, received {len(cw.collector.items)} summaries and {len([i for i in cw.collector.items if i.schema])} full schemas")
    bo_data = DataFrame([i.to_dict() for i in cw.collector.items], copy=False).set_index('busobid')
    if bo_datagrid is None:
        bo_datagrid = DataGrid(bo_data, renderers=renderers, selection_mode='row', column_widths={'name': 280, 'bo_type': 90, 'displayName': 350})
        bo_datagrid.transform([{'type': 'sort', 'columnIndex': 1, 'desc': False}])
    else:
        bo_datagrid.data = bo_data

collect(None, None);

<a id="bo_filter"></a>
### Set up the business object filters

In the following cell, edit the `bo_include_filter` and `bo_exclude_filter` regexes so that the table includes all the business objects that you wish to model.
They can be strings or re.Pattern objects.

In [ ]:
### EDIT THESE TWO VALUES ###

bo_include_filter = r'(?i)ticket$|conf'
bo_exclude_filter = r''

############################################################################################
# If the cell below this one has been run, the table should update if you execute this cell.

status_output.clear_output()
with status_output:
    print(f"{bo_include_filter=!r}")
    print(f"{bo_exclude_filter=!r}")
    collect(bo_include_filter, bo_exclude_filter);


In [ ]:
# Display the collector output in a data grid. The collect() function updates it dynamically

ipywidgets.VBox([status_output, filter_checkbox, bo_datagrid])